<a href="https://colab.research.google.com/github/open-mmlab/mmaction2/blob/master/demo/mmaction2_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMAction2 Tutorial

Welcome to MMAction2! This is the official colab tutorial for using MMAction2. In this tutorial, you will learn
- Perform inference with a MMAction2 recognizer.
- Train a new recognizer with a new dataset.
- Perform spatio-temporal detection.

Let's start!

## Install MMAction2

In [2]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Jun__8_16:49:14_PDT_2022
Cuda compilation tools, release 11.7, V11.7.99
Build cuda_11.7.r11.7/compiler.31442593_0
gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
import os
os.chdir("../../mmaction2")
os.listdir()

['mmaction',
 '.circleci',
 'demo',
 'checkpoints',
 '.readthedocs.yml',
 'setup.cfg',
 'docs',
 'model-index.yml',
 'tmp',
 'README.md',
 'requirements.txt',
 'resources',
 '.gitignore',
 'CITATION.cff',
 'kinetics400_tiny',
 'mmaction2.egg-info',
 '.owners.yml',
 'docs_zh_CN',
 'Untitled.ipynb',
 '.pre-commit-config.yaml',
 'MANIFEST.in',
 'work_dirs',
 '.ipynb_checkpoints',
 'LICENSE',
 'tests',
 '.pylintrc',
 'tutorial_exps',
 'configs',
 'README_en-EN.md',
 'requirements',
 'README_zh-CN.md',
 'docker',
 'kinetics400_tiny.zip',
 'tools',
 'setup.py']

In [4]:
# # install dependencies: (use cu111 because colab has CUDA 11.1)
# !pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# # install mmcv-full thus we could use CUDA operators
# !pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# # Install mmaction2
# !rm -rf mmaction2
# !git clone https://github.com/open-mmlab/mmaction2.git
# %cd mmaction2

# !pip install -e .

# Install some optional requirements
# !pip install -r requirements/optional.txt

In [5]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.13.0+cu117 True
0.24.1
11.7
GCC 9.4


/home/pidl/.local/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


## Perform inference with a MMAction2 recognizer
MMAction2 already provides high level APIs to do inference and training.

In [5]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth \
      -O checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth

mkdir: cannot create directory ‘checkpoints’: File exists
--2022-12-06 20:29:08--  https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.75.20.5
Connecting to download.openmmlab.com (download.openmmlab.com)|47.75.20.5|:443... connected.
HTTP request sent, awaiting response... 200 OK

    The file is already fully retrieved; nothing to do.



In [6]:
from mmaction.apis import inference_recognizer, init_recognizer

# Choose to use a config and initialize the recognizer
config = 'configs/recognition/tsn/tsn_r50_video_inference_1x1x3_100e_kinetics400_rgb.py'
# Setup a checkpoint file to load
checkpoint = 'checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'
# Initialize the recognizer
model = init_recognizer(config, checkpoint, device='cuda:0')

load checkpoint from local path: checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth


In [ ]:
# Use the recognizer to do inference
video = 'demo/demo.mp4'
label = 'tools/data/kinetics/label_map_k400.txt'
results = inference_recognizer(model, video)

labels = open(label).readlines()
labels = [x.strip() for x in labels]
results = [(labels[k[0]], k[1]) for k in results]

In [ ]:
# Let's show the results
for result in results:
    print(f'{result[0]}: ', result[1])

## Train a recognizer on customized dataset

To train a new recognizer, there are usually three things to do:
1. Support a new dataset
2. Modify the config
3. Train a new recognizer

### Support a new dataset

In this tutorial, we gives an example to convert the data into the format of existing datasets. Other methods and more advanced usages can be found in the [doc](/docs/tutorials/new_dataset.md)

Firstly, let's download a tiny dataset obtained from [Kinetics-400](https://deepmind.com/research/open-source/open-source-datasets/kinetics/). We select 30 videos with their labels as train dataset and 10 videos with their labels as test dataset.

In [ ]:
# download, decompress the data
!rm kinetics400_tiny.zip*
!rm -rf kinetics400_tiny
!wget https://download.openmmlab.com/mmaction/kinetics400_tiny.zip
!unzip kinetics400_tiny.zip > /dev/null

In [7]:
# Check the directory structure of the tiny data
# Install tree first
!apt-get -q install tree
!tree kinetics400_tiny

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
kinetics400_tiny
├── kinetics_tiny_train_video.txt
├── kinetics_tiny_val_video.txt
├── train
│   ├── 27_CSXByd3s.mp4
│   ├── 34XczvTaRiI.mp4
│   ├── A-wiliK50Zw.mp4
│   ├── D32_1gwq35E.mp4
│   ├── D92m0HsHjcQ.mp4
│   ├── DbX8mPslRXg.mp4
│   ├── FMlSTTpN3VY.mp4
│   ├── h10B9SVE-nk.mp4
│   ├── h2YqqUhnR34.mp4
│   ├── iRuyZSKhHRg.mp4
│   ├── IyfILH9lBRo.mp4
│   ├── kFC3KY2bOP8.mp4
│   ├── LvcFDgCAXQs.mp4
│   ├── O46YA8tI530.mp4
│   ├── oMrZaozOvdQ.mp4
│   ├── oXy-e_P_cAI.mp4
│   ├── P5M-hAts7MQ.mp4
│   ├── phDqGd0NKoo.mp4
│   ├── PnOe3GZRVX8.mp4
│   ├── R8HXQkdgKWA.mp4
│   ├── RqnKtCEoEcA.mp4
│   ├── soEcZZsBmDs.mp4
│   ├── TkkZPZHbAKA.mp4
│   ├── T_TMNGzVrDk.mp4
│   ├── WaS0qwP46Us.mp4
│   ├── Wh_YPQdH1Zg.mp4
│   ├── WWP5HZJsg-o.mp4
│   ├── xGY2dP0YUjA.mp4
│   ├── yLC9CtWU5ws.mp4
│   └── ZQV4U2KQ370.mp4
└── val
   

In [8]:
# After downloading the data, we need to check the annotation format
!cat kinetics400_tiny/kinetics_tiny_train_video.txt

D32_1gwq35E.mp4 0
iRuyZSKhHRg.mp4 1
oXy-e_P_cAI.mp4 0
34XczvTaRiI.mp4 1
h2YqqUhnR34.mp4 0
O46YA8tI530.mp4 0
kFC3KY2bOP8.mp4 1
WWP5HZJsg-o.mp4 1
phDqGd0NKoo.mp4 1
yLC9CtWU5ws.mp4 0
27_CSXByd3s.mp4 1
IyfILH9lBRo.mp4 1
T_TMNGzVrDk.mp4 1
TkkZPZHbAKA.mp4 0
PnOe3GZRVX8.mp4 1
soEcZZsBmDs.mp4 1
FMlSTTpN3VY.mp4 1
WaS0qwP46Us.mp4 0
A-wiliK50Zw.mp4 1
oMrZaozOvdQ.mp4 1
ZQV4U2KQ370.mp4 0
DbX8mPslRXg.mp4 1
h10B9SVE-nk.mp4 1
P5M-hAts7MQ.mp4 0
R8HXQkdgKWA.mp4 0
D92m0HsHjcQ.mp4 0
RqnKtCEoEcA.mp4 0
LvcFDgCAXQs.mp4 0
xGY2dP0YUjA.mp4 0
Wh_YPQdH1Zg.mp4 0


According to the format defined in [`VideoDataset`](./datasets/video_dataset.py), each line indicates a sample video with the filepath and label, which are split with a whitespace.

### Modify the config

In the next step, we need to modify the config for the training.
To accelerate the process, we finetune a recognizer using a pre-trained recognizer.

In [9]:
from mmcv import Config
cfg = Config.fromfile('configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb.py')

Given a config that trains a TSN model on kinetics400-full dataset, we need to modify some values to use it for training TSN on Kinetics400-tiny dataset.


In [10]:
from mmcv.runner import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'VideoDataset'
cfg.data_root = 'kinetics400_tiny/train/'
cfg.data_root_val = 'kinetics400_tiny/val/'
cfg.ann_file_train = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.ann_file_val = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.ann_file_test = 'kinetics400_tiny/kinetics_tiny_val_video.txt'

cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.data.test.data_prefix = 'kinetics400_tiny/val/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.data.train.data_prefix = 'kinetics400_tiny/train/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.data.val.data_prefix = 'kinetics400_tiny/val/'

# The flag is used to determine whether it is omnisource training
cfg.setdefault('omnisource', False)
# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 2
# We can use the pre-trained TSN model
cfg.load_from = 'checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

# Set up working dir to save files and logs.
cfg.work_dir = 'tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.data.videos_per_gpu = cfg.data.videos_per_gpu // 16
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
cfg.total_epochs = 10

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 5
# We can set the log print interval to reduce the the times of printing log
cfg.log_config.interval = 5

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# Save the best
cfg.evaluation.save_best='auto'


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Config:
model = dict(
    type='Recognizer2D',
    backbone=dict(
        type='ResNet',
        pretrained='torchvision://resnet50',
        depth=50,
        norm_eval=False),
    cls_head=dict(
        type='TSNHead',
        num_classes=2,
        in_channels=2048,
        spatial_type='avg',
        consensus=dict(type='AvgConsensus', dim=1),
        dropout_ratio=0.4,
        init_std=0.01),
    train_cfg=None,
    test_cfg=dict(average_clips=None))
optimizer = dict(type='SGD', lr=7.8125e-05, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=dict(max_norm=40, norm_type=2))
lr_config = dict(policy='step', step=[40, 80])
total_epochs = 10
checkpoint_config = dict(interval=5)
log_config = dict(interval=5, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = 'checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'
resume_from = None
workflow = [('train', 1)]
opencv_num_threads = 0
mp_start_method = 'fo

### Train a new recognizer

Finally, lets initialize the dataset and recognizer, then train a new recognizer!

In [11]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the recognizer
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate=True)

2022-12-06 20:30:00,808 - mmaction - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.weight', 'fc.bias'}
2022-12-06 20:30:00,866 - mmaction - INFO - load checkpoint from local path: checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth


load checkpoint from torchvision path: torchvision://resnet50


2022-12-06 20:30:00,955 - mmaction - WARNING - The model and loaded state dict do not match exactly

size mismatch for cls_head.fc_cls.weight: copying a param with shape torch.Size([400, 2048]) from checkpoint, the shape in current model is torch.Size([2, 2048]).
size mismatch for cls_head.fc_cls.bias: copying a param with shape torch.Size([400]) from checkpoint, the shape in current model is torch.Size([2]).
2022-12-06 20:30:00,961 - mmaction - INFO - Start running, host: pidl@pidl, work_dir: /home/pidl/working/mmtools/CoCoVa/mmaction2/tutorial_exps
2022-12-06 20:30:00,962 - mmaction - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook              

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 6.2 task/s, elapsed: 2s, ETA:     0s

2022-12-06 20:30:31,882 - mmaction - INFO - Evaluating top_k_accuracy ...
2022-12-06 20:30:31,884 - mmaction - INFO - 
top1_acc	0.7000
top5_acc	1.0000
2022-12-06 20:30:31,885 - mmaction - INFO - Evaluating mean_class_accuracy ...
2022-12-06 20:30:31,886 - mmaction - INFO - 
mean_acc	0.7000
2022-12-06 20:30:32,324 - mmaction - INFO - Now best checkpoint is saved as best_top1_acc_epoch_5.pth.
2022-12-06 20:30:32,325 - mmaction - INFO - Best top1_acc is 0.7000 at 5 epoch.
2022-12-06 20:30:32,326 - mmaction - INFO - Epoch(val) [5][5]	top1_acc: 0.7000, top5_acc: 1.0000, mean_class_accuracy: 0.7000
2022-12-06 20:30:35,869 - mmaction - INFO - Epoch [6][5/15]	lr: 7.813e-05, eta: 0:00:27, time: 0.707, data_time: 0.516, memory: 1654, top1_acc: 0.7000, top5_acc: 1.0000, loss_cls: 0.6413, loss: 0.6413, grad_norm: 12.5305
2022-12-06 20:30:36,804 - mmaction - INFO - Epoch [6][10/15]	lr: 7.813e-05, eta: 0:00:25, time: 0.187, data_time: 0.001, memory: 1654, top1_acc: 0.5000, top5_acc: 1.0000, loss_cls

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 6.4 task/s, elapsed: 2s, ETA:     0s

2022-12-06 20:31:01,658 - mmaction - INFO - Evaluating top_k_accuracy ...
2022-12-06 20:31:01,660 - mmaction - INFO - 
top1_acc	0.8000
top5_acc	1.0000
2022-12-06 20:31:01,661 - mmaction - INFO - Evaluating mean_class_accuracy ...
2022-12-06 20:31:01,662 - mmaction - INFO - 
mean_acc	0.8000
2022-12-06 20:31:01,701 - mmaction - INFO - The previous best checkpoint /home/pidl/working/mmtools/CoCoVa/mmaction2/tutorial_exps/best_top1_acc_epoch_5.pth was removed
2022-12-06 20:31:02,117 - mmaction - INFO - Now best checkpoint is saved as best_top1_acc_epoch_10.pth.
2022-12-06 20:31:02,119 - mmaction - INFO - Best top1_acc is 0.8000 at 10 epoch.
2022-12-06 20:31:02,120 - mmaction - INFO - Epoch(val) [10][5]	top1_acc: 0.8000, top5_acc: 1.0000, mean_class_accuracy: 0.8000


### Understand the log
From the log, we can have a basic understanding the training process and know how well the recognizer is trained.

Firstly, the ResNet-50 backbone pre-trained on ImageNet is loaded, this is a common practice since training from scratch is more cost. The log shows that all the weights of the ResNet-50 backbone are loaded except the `fc.bias` and `fc.weight`.

Second, since the dataset we are using is small, we loaded a TSN model and finetune it for action recognition.
The original TSN is trained on original Kinetics-400 dataset which contains 400 classes but Kinetics-400 Tiny dataset only have 2 classes. Therefore, the last FC layer of the pre-trained TSN for classification has different weight shape and is not used.

Third, after training, the recognizer is evaluated by the default evaluation. The results show that the recognizer achieves 100% top1 accuracy and 100% top5 accuracy on the val dataset,
 
Not bad!

## Test the trained recognizer

After finetuning the recognizer, let's check the prediction results!

In [12]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel

# Build a test dataloader
dataset = build_dataset(cfg.data.test, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[0])
outputs = single_gpu_test(model, data_loader)

eval_config = cfg.evaluation
eval_config.pop('interval')
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 1.1 task/s, elapsed: 9s, ETA:     0s
Evaluating top_k_accuracy ...

top1_acc	0.8000
top5_acc	1.0000

Evaluating mean_class_accuracy ...

mean_acc	0.8000
top1_acc: 0.8000
top5_acc: 1.0000
mean_class_accuracy: 0.8000


/home/pidl/working/mmtools/CoCoVa/mmaction2/mmaction/datasets/base.py:166: UserWarning: Option arguments for metrics has been changed to `metric_options`, See 'https://github.com/open-mmlab/mmaction2/pull/286' for more details
  warnings.warn(


## Perform Spatio-Temporal Detection
Here we first install MMDetection.

In [ ]:
# Git clone mmdetection repo
%cd ..
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

# install mmdet
!pip install -e .
%cd ../mmaction2

Download a video to `demo` directory in MMAction2.

In [13]:
!wget https://download.openmmlab.com/mmaction/dataset/sample/1j20qq1JyX4.mp4 -O demo/1j20qq1JyX4.mp4

--2022-12-06 20:33:42--  https://download.openmmlab.com/mmaction/dataset/sample/1j20qq1JyX4.mp4
Resolving download.openmmlab.com (download.openmmlab.com)... 47.75.20.5
Connecting to download.openmmlab.com (download.openmmlab.com)|47.75.20.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4864186 (4.6M) [video/mp4]
Saving to: ‘demo/1j20qq1JyX4.mp4’

demo/1j20qq1JyX4.mp 100%[===================>]   4.64M  1.64MB/s    in 2.8s    

2022-12-06 20:33:47 (1.64 MB/s) - ‘demo/1j20qq1JyX4.mp4’ saved [4864186/4864186]



Run spatio-temporal demo.

In [18]:
!python demo/demo_spatiotemporal_det.py --video demo/1j20qq1JyX4.mp4

/home/pidl/anaconda3/envs/mmtool/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
load checkpoint from local path: checkpoints/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
Performing Human Detection for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 217/217, 7.4 task/s, elapsed: 29s, ETA:     0sload checkpoint from local path: checkpoints/slowonly_omnisource_pretrained_r101_8x8x1_20e_ava_rgb_20201217-16378594.pth
Performing SpatioTemporal Action Detection for each clip
[>>>>>>>>>>>>>>>>>>>>>>>       ] 167/217, 7.2 task/s, elapsed: 23s, ETA:     7sPerforming visualization
Moviepy - Building video demo/stdet_

In [1]:
# Check the video
from IPython.display import HTML
from base64 import b64encode
mp4 = open('demo/stdet_demo.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

FileNotFoundError: [Errno 2] No such file or directory: 'demo/stdet_demo.mp4'